#### First get the candidate model parts (Head and Feature Extractor)

In [5]:
#Only model weights in GPU: 7.9 GB
#Batch of 4 images pairs forward pass: +13.5GB
#Batch of 4 images pairs backward pass: +5GB
#Fordward pass time GPU batch 4 pairs: 300 ms
#forward + Backward  pass time CPU batch 4 pairs: 13 segs + 40 segs

In [1]:
import os
import torch
import torch.nn as nn
import time
from torch.distributed.rpc import init_rpc
from drecg.feature_extraction.distributed import define_model_for_tune


os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '29500'
init_rpc('worker', rank=0, world_size=1)


devices_list = [torch.device("cuda:0"), torch.device("cuda:1"),torch.device("cuda:2"),torch.device("cuda:3")]
model_pipe = define_model_for_tune(devices_list, microbatch_num=2)

loss_fn = nn.BCEWithLogitsLoss()
y_true = torch.ones(16, 1, dtype=torch.float32).to(devices_list[-1])

model_pipe.train()
adam_w = torch.optim.AdamW(model_pipe.parameters(), lr=1e-3)



config.json not found in HuggingFace Hub.


In [3]:
model_pipe.train()
for i in range(10):
    init_time = time.time()
    dummy_img_a = torch.rand(16, 3, 224, 224).to(devices_list[0])
    dummy_img_b = torch.rand(16, 3, 224, 224).to(devices_list[0])
    adam_w.zero_grad()
    out = model_pipe((dummy_img_a, dummy_img_b))
    #out = out.local_value()
    loss = loss_fn(out, y_true)
    loss.backward()
    adam_w.step()
    total_time_segs = time.time() - init_time
    print("iter: {}, loss: {}, time: {} segs".format(i, loss.item(), total_time_segs))

iter: 0, loss: 0.04550361633300781, time: 2.6270127296447754 segs
iter: 1, loss: 0.034522946923971176, time: 2.9066762924194336 segs
iter: 2, loss: 0.02579524740576744, time: 2.9080464839935303 segs
iter: 3, loss: 0.02561810426414013, time: 2.916492223739624 segs
iter: 4, loss: 0.018328363075852394, time: 2.922459363937378 segs
iter: 5, loss: 0.008190635591745377, time: 2.9422311782836914 segs
iter: 6, loss: 0.01341742929071188, time: 2.919269323348999 segs
iter: 7, loss: 0.012847553007304668, time: 2.926701068878174 segs
iter: 8, loss: 0.0108527522534132, time: 2.926661491394043 segs
iter: 9, loss: 0.009601861238479614, time: 2.9174177646636963 segs
